In [1]:
import os
import re
import time
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta


# ігнорування попередження SettingWithCopyWarning
pd.set_option('mode.chained_assignment', None)  


In [3]:
list_columns =   [
        "Date",
        "HomeTeam",
        "AwayTeam",
        "rez_h_full",
        "rez_a_full",
        "rez_h_first",
        "rez_a_first",
        "h_first",
        "h_full",
        "a_first",
        "a_full",
    ] 
# funcs

def get_team_results(
                    h_team_name: str,
                    a_team_name: str,
                    matches_before: pd.DataFrame,
                    ) -> tuple[int, int, int, int] :
    """
    отримання результатів команд, останнього матчу
    повертає список з 4 чисел, різниці забитих мячів
      - дом. команди в першому таймі,
      - дом. команди в повному матчі,
      - гост. команди в першому таймі,
      - гост. команди в повному матчі,
    """
    h_team_match = matches_before[                            
                        (matches_before["HomeTeam"] == h_team_name) | 
                        (matches_before["AwayTeam"] == h_team_name)
                    ].tail(1) 
    
    a_team_match = matches_before[
                        (matches_before["HomeTeam"] == a_team_name) |
                        (matches_before["AwayTeam"] == a_team_name)
                    ].tail(1) 
    
#     print()
#     print(h_team_match)
#     print()
#     print(a_team_match)
#     print()

    h_first = (
                    h_team_match.at[h_team_match.index[0], "HTHG"] - 
                    h_team_match.at[h_team_match.index[0], "HTAG"]
                )


    h_full = (
                    h_team_match.at[h_team_match.index[0], "FTHG"] - 
                    h_team_match.at[h_team_match.index[0], "FTAG"]
                )

    h_first = h_first if h_team_name in h_team_match["HomeTeam"].values else -h_first
    h_full = h_full if h_team_name in h_team_match["HomeTeam"].values else -h_full


    a_first = (
                    a_team_match.at[a_team_match.index[0], "HTHG"] - 
                    a_team_match.at[a_team_match.index[0], "HTAG"]
                )

    a_full = (
                    a_team_match.at[a_team_match.index[0], "FTHG"] - 
                    a_team_match.at[a_team_match.index[0], "FTAG"]
                )
    a_first = a_first if a_team_name in a_team_match["HomeTeam"].values else -a_first
    a_full = a_full if a_team_name in a_team_match["HomeTeam"].values else -a_full
    
    return (
        h_first,
        h_full,
        a_first,
        a_full,
    )

In [6]:
# конвертування строчки дати в формат дати 
d, m, y = [int(x) for x in df.at[0, "Date"].split("/")]
a = date(y, m, d)
a
# date(2023, 5, 9).isoweekday() == 2  # первірка дати на вівторок

AttributeError: 'datetime.date' object has no attribute 'split'

In [147]:
# df = pd.read_csv("./data/eo_21_22.csv")
# df = pd.read_csv("./data/sp1_21_22.csv")
df = pd.read_csv("./data/sp2_19_20.csv")
df.loc[:, "Date"] =  [date.fromisoformat("-".join(x.split("/")[::-1]))  for x in df.loc[:, "Date"].values]

df

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,SP2,2019-08-17,17:00,Lugo,Extremadura UD,0,0,D,0,0,...,1.61,-0.25,1.90,1.95,1.91,1.99,1.94,2.02,1.89,1.96
1,SP2,2019-08-17,17:00,Santander,Malaga,0,1,A,0,0,...,1.47,-0.50,1.95,1.90,2.04,1.85,2.06,1.95,1.99,1.86
2,SP2,2019-08-17,18:00,Almeria,Albacete,3,0,H,0,0,...,1.55,-0.25,1.93,1.93,1.93,1.97,1.98,1.98,1.92,1.93
3,SP2,2019-08-17,19:00,Elche,Fuenlabrada,0,2,A,0,2,...,1.48,-0.25,1.88,1.98,1.89,2.01,1.93,2.21,1.85,2.00
4,SP2,2019-08-17,20:00,Vallecano,Mirandes,2,2,D,2,1,...,1.82,-1.00,1.98,1.88,2.03,1.87,2.06,1.96,1.97,1.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,SP2,2020-07-20,20:00,Numancia,Tenerife,2,1,H,1,0,...,1.90,-0.75,1.83,2.02,1.87,2.03,1.88,2.12,1.83,2.02
458,SP2,2020-07-20,20:00,Santander,Vallecano,1,2,A,1,0,...,2.18,1.00,2.02,1.83,2.06,1.84,2.11,1.87,2.03,1.81
459,SP2,2020-07-20,20:00,Sp Gijon,Huesca,0,1,A,0,0,...,1.87,0.00,2.02,1.83,2.02,1.88,2.18,1.90,2.02,1.83
460,SP2,2020-07-20,20:00,Zaragoza,Ponferradina,2,1,H,1,0,...,1.93,0.00,1.70,2.10,1.72,2.21,1.87,2.21,1.74,2.12


In [148]:
df_data = df.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "HTHG", "HTAG"]]
df_data

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HTHG,HTAG
0,2019-08-17,Lugo,Extremadura UD,0,0,0,0
1,2019-08-17,Santander,Malaga,0,1,0,0
2,2019-08-17,Almeria,Albacete,3,0,0,0
3,2019-08-17,Elche,Fuenlabrada,0,2,0,2
4,2019-08-17,Vallecano,Mirandes,2,2,2,1
...,...,...,...,...,...,...,...
457,2020-07-20,Numancia,Tenerife,2,1,1,0
458,2020-07-20,Santander,Vallecano,1,2,1,0
459,2020-07-20,Sp Gijon,Huesca,0,1,0,0
460,2020-07-20,Zaragoza,Ponferradina,2,1,1,0


In [149]:
rez_data = []
for ind in df_data.index:
    current_match = df_data.loc[ind]
    matches_before = df_data.loc[:ind-1]
    h_team_name = current_match["HomeTeam"]
    a_team_name = current_match["AwayTeam"]
    if ( 
        (h_team_name in matches_before["HomeTeam"].values or
                 h_team_name in matches_before["AwayTeam"].values) and 
        (a_team_name in matches_before["HomeTeam"].values or
                 a_team_name in matches_before["AwayTeam"].values)  
    ):
        h_first, h_full, a_first, a_full = get_team_results(
                                                     h_team_name,
                                                     a_team_name,
                                                    matches_before
                                                    )
        
        print(ind, h_team_name, a_team_name)
        print()
        print(matches_before.tail(15))
        print()
        
        print(h_full, h_first)
        
        print()
        print(a_full, a_first)
        
        data_row = [
            *current_match.values,
            h_first,
            h_full,
            a_first,
            a_full
        ]
#         print(data_row)
        rez_data.append(data_row)
#         raise

11 Albacete Girona

          Date    HomeTeam        AwayTeam  FTHG  FTAG  HTHG  HTAG
0   2019-08-17        Lugo  Extremadura UD     0     0     0     0
1   2019-08-17   Santander          Malaga     0     1     0     0
2   2019-08-17     Almeria        Albacete     3     0     0     0
3   2019-08-17       Elche     Fuenlabrada     0     2     0     2
4   2019-08-17   Vallecano        Mirandes     2     2     2     1
5   2019-08-17    Zaragoza        Tenerife     2     0     1     0
6   2019-08-18   La Coruna          Oviedo     3     2     1     0
7   2019-08-18    Numancia        Alcorcon     0     1     0     0
8   2019-08-18      Girona        Sp Gijon     1     1     0     0
9   2019-08-18       Cadiz    Ponferradina     3     1     0     1
10  2019-08-18  Las Palmas          Huesca     0     1     0     0

-3 0

0 0
12 Oviedo Lugo

          Date    HomeTeam        AwayTeam  FTHG  FTAG  HTHG  HTAG
0   2019-08-17        Lugo  Extremadura UD     0     0     0     0
1   2019-08-17 

253 Numancia Sp Gijon

           Date     HomeTeam        AwayTeam  FTHG  FTAG  HTHG  HTAG
238  2020-01-05    Vallecano          Girona     1     0     1     0
239  2020-01-05       Oviedo          Malaga     1     1     1     0
240  2020-01-07     Zaragoza        Sp Gijon     2     0     1     0
241  2020-01-14     Albacete     Fuenlabrada     1     1     1     0
242  2020-01-14     Sp Gijon           Elche     1     0     0     0
243  2020-01-14       Malaga    Ponferradina     1     0     1     0
244  2020-01-14   Las Palmas        Zaragoza     0     1     0     0
245  2020-01-15     Alcorcon        Numancia     2     2     0     0
246  2020-01-15       Huesca        Tenerife     2     1     2     1
247  2020-01-15    Vallecano            Lugo     1     0     1     0
248  2020-01-15        Cadiz        Mirandes     3     3     1     1
249  2020-01-16    La Coruna       Santander     2     1     1     1
250  2020-01-16       Girona  Extremadura UD     3     1     2     0
251  2020-0

434 Numancia Ponferradina

           Date        HomeTeam        AwayTeam  FTHG  FTAG  HTHG  HTAG
419  2020-07-07     Fuenlabrada       Santander     1     0     0     0
420  2020-07-07  Extremadura UD           Cadiz     0     1     0     1
421  2020-07-08          Malaga       La Coruna     1     0     0     0
422  2020-07-08          Oviedo      Las Palmas     2     1     1     0
423  2020-07-08          Huesca        Alcorcon     2     1     2     1
424  2020-07-08    Ponferradina            Lugo     0     1     0     0
425  2020-07-09        Albacete        Sp Gijon     1     1     0     1
426  2020-07-09       Vallecano        Numancia     3     2     2     0
427  2020-07-09          Girona         Almeria     1     0     0     0
428  2020-07-09        Tenerife        Zaragoza     1     1     1     1
429  2020-07-11       Santander          Huesca     1     0     1     0
430  2020-07-11      Las Palmas        Mirandes     1     0     0     0
431  2020-07-11           Cadiz     F

In [150]:
clean_data = pd.DataFrame(
                        data=rez_data,
                        columns=list_columns
            )
clean_data#.head(25)

,Date,HomeTeam,AwayTeam,rez_h_full,rez_a_full,rez_h_first,rez_a_first,h_first,h_full,a_first,a_full
0,2019-08-23,Albacete,Girona,1,0,0,0,0,-3,0,0
1,2019-08-24,Oviedo,Lugo,1,1,1,0,-1,-1,0,0
2,2019-08-24,Santander,Almeria,1,1,1,0,0,-1,0,3
3,2019-08-24,Mirandes,Cadiz,1,2,0,0,-1,0,-1,2
4,2019-08-24,Alcorcon,Elche,1,2,1,0,0,1,-2,-2
...,...,...,...,...,...,...,...,...,...,...,...
446,2020-07-20,Numancia,Tenerife,2,1,1,0,0,-3,0,-1
447,2020-07-20,Santander,Vallecano,1,2,1,0,-1,-1,0,0
448,2020-07-20,Sp Gijon,Huesca,0,1,0,0,-2,-2,0,3
449,2020-07-20,Zaragoza,Ponferradina,2,1,1,0,-2,-3,0,1


In [151]:
start_day = clean_data["Date"].values[0]
end_day = clean_data["Date"].values[-1]

list_days = []
while start_day <= end_day:
    list_days.append(start_day)
    start_day += timedelta(days=1)
    
only_tuesdays = list(
                    filter(
                        lambda day: day.isoweekday() == 2,
                        list_days
                    )
                )

# only_tuesdays

In [152]:
"""
нічья перший тайм, якщо h_first, a_first = 0, 0 | 1, 1
"""
clean_data["index"] = clean_data.index
full_df_np = clean_data[[
            "h_first", "h_full",
            "a_first", "a_full",
            "rez_h_full", "rez_a_full", "rez_h_first", "rez_a_first",
            "index"
    ]].to_numpy()

for h in range(-4, 4): 
    for a in range(-4, 4): 
        if h != a : continue
        filtred_full = full_df_np[
            (full_df_np[:, 0] == h)
            & (full_df_np[:, 2] == a)
        ]

        win = filtred_full[
                filtred_full[:, 6] == filtred_full[:, 7]

        ]
        proc = win.shape[0] / filtred_full.shape[0] if filtred_full.shape[0] > 0 else 0
        if proc < 0.0: continue
        print(
            f"всього - {filtred_full.shape[0]:3d} , перемог - {win.shape[0]:3d} => {proc:.3f} |||"
            f" {h}  {a}"
        )
#         raise

всього -   0 , перемог -   0 => 0.000 ||| -4  -4
всього -   0 , перемог -   0 => 0.000 ||| -3  -3
всього -   1 , перемог -   0 => 0.000 ||| -2  -2
всього -  20 , перемог -  12 => 0.600 ||| -1  -1
всього -  94 , перемог -  47 => 0.500 ||| 0  0
всього -  19 , перемог -  11 => 0.579 ||| 1  1
всього -   0 , перемог -   0 => 0.000 ||| 2  2
всього -   0 , перемог -   0 => 0.000 ||| 3  3


In [106]:
rez_dict = {
    "win" : 0,
    "lose" : 0
}
for tuesday in only_tuesdays:
    some_rez = np.array([], dtype=int).reshape(0,9)
    
    work_df = clean_data[
        (clean_data["Date"] >= tuesday - timedelta(days=14)) &
        (clean_data["Date"] < tuesday) 
    ] 
    
     
    target_df = clean_data[
        (clean_data["Date"] >= tuesday)  &
        (clean_data["Date"] < tuesday + timedelta(days=7)) 
    ]
    
    # минулі результати
    work_df["index"] = work_df.index
    work_df_np = work_df[[
                            "h_first", "h_full",
                            "a_first", "a_full",
                            "rez_h_full", "rez_a_full", "rez_h_first", "rez_a_first",
                            "index"
                ]].to_numpy()
    
   
    # майбутні результати
    target_df["index"] = target_df.index
    target_df_np = target_df[[
                            "h_first", "h_full",
                            "a_first", "a_full",
                            "rez_h_full", "rez_a_full", "rez_h_first", "rez_a_first",
                            "index"
                ]].to_numpy()
    
    
    some_range = np.arange(-5, 5, 1)
    
    # пошук в минулих результатах
    # поки не працює!!!
    """
    for h_first in some_range:
        for h_full in some_range:
            for a_first in some_range:
                for a_full in some_range:
                    filtered_df_np = work_df_np[
                        (work_df_np[:, 0] == h_first) &
                        (work_df_np[:, 1] == h_full) &
                        (work_df_np[:, 2] == a_first) &
                        (work_df_np[:, 3] == a_full) 
                    ]
                    
#                     if filtered_df_np.shape[0] < 2 : continue
                    draw_first = filtered_df_np[
                                                filtered_df_np[:, 6] == filtered_df_np[:, 7]
                                            ]
                    tb_1_5_first = filtered_df_np[
                                                filtered_df_np[:, 6] + filtered_df_np[:, 7] > 1.5 
                                            ]
        
                    tm_1_5_first = filtered_df_np[
                                                filtered_df_np[:, 6] + filtered_df_np[:, 7] < 1.5 
                                            ]
                       
                       
#                     if draw_first.shape[0] >= 1: 
#                         some_rez = np.concatenate((some_rez,  draw_first))
                        
#                     if tb_1_5_first.shape[0] >= 1: 
#                         some_rez = np.concatenate((some_rez,  tb_1_5_first))
                        
                    if tm_1_5_first.shape[0] >= 1: 
                        some_rez = np.concatenate((some_rez,  tm_1_5_first))
    
#     if some_rez.shape[0] == 0: continue
    """
    
    
    # пошук в майбутніх результатах
    for data in target_df_np:
        h_first = data[0]
        h_full = data[1]
        a_first = data[2]
        a_full = data[3]
 
        filtered_df_np = work_df_np[
            (work_df_np[:, 0] == h_first) 
            & 
            (work_df_np[:, 1] == h_full) 
            &
            (work_df_np[:, 2] == a_first) 
            & (work_df_np[:, 3] == a_full) 
        ]
    
        if (
            filtered_df_np.shape[0] >= 1
            and (filtered_df_np[:, 6] == filtered_df_np[:, 7]).all()
#             and (filtered_df_np[:, 6] + filtered_df_np[:, 7] > 1.5).all()
            ):
#             [print(x) for x in some_rez]
#             print(tuesday)
#             print(work_df)
#             print(target_df)
            print("="*20)
            print(data)
            print()

            print(filtered_df_np )
            
        
            if  (data[6] == data[7]):
#             if  (data[6] + data[7] > 1.5):
                rez_dict["win"] += 1
                print("good")
            else:
                rez_dict["lose"] += 1
                print("fuck")
#             print("good") if  (filtered_df_np[:, 6] + filtered_df_np[:, 7] > 1.5).all() else print("fuck")
#             print("good") if  (filtered_df_np[:, 6] + filtered_df_np[:, 7] < 1.5).all() else print("fuck")
            print()
            print()
            print(work_df_np)
            print()
            
            print(target_df_np)
            
            raise
            
print(rez_dict)
    
    


[ 0  0  0  0  0  1  0  1 12]

[[0 0 0 0 0 0 0 0 2]]
fuck


[[-1  0 -1  0  1  1  1  1  0]
 [ 0 -3  1  0  0  1  0  0  1]
 [ 0  0  0  0  0  0  0  0  2]
 [ 0  0  1  1  1  1  1  0  3]
 [ 0  0  2  2  1  1  0  0  4]
 [-2 -2 -1 -3  1  0  0  0  5]
 [ 1  1  0  0  1  0  1  0  6]
 [ 1  0  0  3  3  3  0  1  7]
 [-1 -1  1  3  0  1  0  0  8]
 [ 0  0 -1 -1  0  0  0  0  9]]

[[ 0  1  0  0  1  0  1  0 10]
 [-1  0  0 -1  3  0  2  0 11]
 [ 0  0  0  0  0  1  0  1 12]
 [-1 -1  0  1  1  1  1  1 13]
 [ 0  1 -1  0  1  0  1  0 14]
 [ 0  0  1  0  0  1  0  0 15]
 [ 0  0  0 -1  2  1  2  1 16]
 [ 0  0  0  0  2  3  1  0 17]
 [ 0 -1  1  0  4  0  3  0 18]
 [ 1  1  0  0  2  2  0  0 19]]


RuntimeError: No active exception to reraise

In [47]:
filtered_df_np[:, 6] == filtered_df_np[:, 7]

array([ True])

In [53]:
clean_data.loc[349]

Date            2022-05-12
HomeTeam       Real Madrid
AwayTeam           Levante
rez_h_full               6
rez_a_full               0
rez_h_first              4
rez_a_first              0
h_first                 -1
h_full                  -1
a_first                  0
a_full                   1
Name: 349, dtype: object

In [84]:
a = np.array([]).reshape(0,2)
# b = np.array([[2,3], [4.5]])
np.concatenate((a,  [[2,3], [4,5]]))


array([[2., 3.],
       [4., 5.]])